This notebook looks at running DMD using the [modred](https://pypi.python.org/pypi/modred)  on the bubbles videos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import modred as mr

In [ ]:
from skimage import io

In [ ]:
path = '../Test1/'

In [ ]:
# read the videos into a collection
video = io.imread_collection(path+'*.tif')

In [ ]:
len(video)

In [ ]:
T = 100 # decide how many images to read (On a small machine big T this may overload your RAM!!! Tested on EC2 instance with 16GB RAM)

In [ ]:
short_video = np.array(video[:T])

In [ ]:
# reshape in # features x # samples
data = short_video.reshape(T,800*1280).T

In [ ]:
# data = [frame.reshape(800*1280) for frame in video[:10]]

In [ ]:
data.shape

In [ ]:
# DMDHandles is the approach which can work with any type of vectors and inner product, but too involved for simple arrays
# DMD = mr.DMDHandles(mr.InnerProductBlock(100))
# DMD_eigvals, correlation_mat_eigvals = DMD.compute_decomp(data)[0, 3]

We will use the `modred.compute _DMD_matrices_snaps_method` which can work on numpy arrays:

In [ ]:
help(mr.compute_DMD_matrices_snaps_method)

We need to specify the numbers of the modes as a second parameter.

In [ ]:
%%time
exact_modes, proj_modes, spectral_coeffs, eigvals  = mr.compute_DMD_matrices_snaps_method(data.astype('float'),list(range(5)))

In [ ]:
print(exact_modes.shape)
print(proj_modes.shape)

In [ ]:
eigvals # not ordered?

In [ ]:
plt.plot(np.real(spectral_coeffs),np.imag(spectral_coeffs),'ro')
plt.title('Spectral Coeffs')

In [ ]:
plt.figure(figsize = (15,10))
plt.imshow(np.array(np.real(exact_modes)[:,0]).reshape((800,1280)),cmap = 'gray')
plt.title('First Exact Mode')
plt.colorbar()

In [ ]:
#### Extra toy examples for testing

In [ ]:
# POD toy example

num_vecs = 30
# Arbitrary data
vecs = np.random.random((100, num_vecs))
num_modes = 5
modes, eig_vals = mr.compute_POD_matrices_snaps_method(
    vecs, list(range(num_modes)))

In [ ]:
# DMD toy example

num_vecs = 10
# Arbitrary data
vecs = np.random.random((1024000, num_vecs))
num_modes = 5
res = mr.compute_DMD_matrices_snaps_method(
    vecs, list(range(num_modes)))